In [40]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 KB 2.8 MB/s eta 0:00:00a 0:00:01


In [151]:
import zipfile
import os
from io import BytesIO
from datetime import datetime
import unidecode
import re

import polars as pl
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

In [2]:
BASE_URL = os.environ.get("BASE_URL")
print(BASE_URL)

/home/prueba/cgr_corrupcion/


In [3]:
# datasets
folder_data = BASE_URL + 'data/'

datasets = {
    'ds_airhsp' : 'DS01_AIRHSP.zip',
    'ds_seace' : 'DS02_SEACE.zip',
    'ds_siaf' : 'DS03_ORDENES_SIAF.zip',
    'ds_empresas' : 'DS04_REPRESENTANTES_EMPRESAS.zip',
    'ds_servir' : 'DS05_SANCIONADOS_SERVIR.zip',
    'ds_gstos' : 'DS06_GASTOS_EJECUTORA.zip',
    'ds_proveedor' : 'DS07_PROVEEDORES_PROYECTOS.zip',
    'ds_proyectos' : 'DS07_PROYECTOS_INVERSION.zip',
    'ds_inco' : 'DS08_INCO.zip'
}

In [4]:
def get_dataset(id_dataset,id_elem):
    archivo_zip = datasets[id_dataset]
    ruta_zip = os.path.join(folder_data, archivo_zip)
    df = pl.DataFrame()
    
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        print(f"\nTamaños de archivos CSV en {archivo_zip}:")
        # Iterar sobre los archivos dentro del zip
        list_archivo = []
        for nombre_archivo in zip_ref.namelist():
            # Verificar si el archivo es un archivo CSV
            
            if nombre_archivo.endswith('.csv'):
                list_archivo.append(nombre_archivo)
            
        print(list_archivo)
        print(list_archivo[id_elem])
        with zip_ref.open(list_archivo[id_elem]) as archivo_csv:
            # df = pl.read_csv(archivo_csv, ignore_errors=True)
            df = pd.read_csv(archivo_csv)
            
        # Obtener el tamaño del archivo en bytes
        # tamano_bytes = zip_ref.getinfo(nombre_archivo).file_size

        # # Convertir el tamaño a megabytes para mayor legibilidad
        # tamano_mb = tamano_bytes / (1024 ** 3)

        # # Imprimir el nombre del archivo y su tamaño
        # print(f'{nombre_archivo}: {tamano_mb:.2f} GB')
            # break
    return df

### PROVEEDORES

In [184]:
%%time

proveedor_data = get_dataset("ds_empresas",0)


Tamaños de archivos CSV en DS04_REPRESENTANTES_EMPRESAS.zip:
['DS04_REPRESENTANTES.csv']
DS04_REPRESENTANTES.csv


/tmp/ipykernel_1576940/1695319871.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_csv)


CPU times: user 1min 53s, sys: 7.84 s, total: 2min 1s
Wall time: 2min 3s


In [185]:
proveedor_data.columns

Index(['RUC_PROVEEDOR', 'NTDI_CODIGO', 'TIPO_DOCUMENTO', 'NUMERO_DOCUMENTO',
       'TIPO_REP', 'CARGO', 'PORCENTAJE', 'VALORTOTALACCIONES', 'NU_ACCIONES',
       'FECHA_REGISTRO', 'FH_INGRESO', 'FH_SALIDA', 'ESTADO_REP', 'FUENTE',
       'ACT_PRIMARIA', 'ESTADO_PROVEEDOR', 'CONDICION_PROVEEDOR', 'DDP_NOMVIA',
       'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'],
      dtype='object')

In [186]:
proveedor_data.head(2)

,RUC_PROVEEDOR,NTDI_CODIGO,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,TIPO_REP,CARGO,PORCENTAJE,VALORTOTALACCIONES,NU_ACCIONES,FECHA_REGISTRO,FH_INGRESO,FH_SALIDA,ESTADO_REP,FUENTE,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DDP_NOMVIA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,10100275560,1,DNI,10027556.0,REP. SUNAT,MISMA PERSONA,NaN,NaN,NaN,NaN,13/03/20133,31/12/99999,ACTIVO VIGENTE,SUNAT,NaN,ACTIVO,NO HABIDO,AVENIDA TUPAC AMARU ASENTAMIENTO HUMANO VELA...,UCAYALI,CORONEL PORTILLO,CALLERIA
1,10100793370,1,DNI,10079337.0,REP. SUNAT,MISMA PERSONA,NaN,NaN,NaN,NaN,11/11/20055,31/12/99999,ACTIVO VIGENTE,SUNAT,NaN,ACTIVO,HABIDO,CALLE LAS AMERICAS URBANIZACION JOSE LISHNER...,TUMBES,TUMBES,TUMBES


In [187]:
# proveedor_data['DEPARTAMENTO'] = proveedor_data['DEPARTAMENTO'].astype(str).apply(unidecode.unidecode)
# proveedor_data['PROVINCIA'] = proveedor_data['PROVINCIA'].astype(str).apply(unidecode.unidecode)
# proveedor_data['DISTRITO'] = proveedor_data['DISTRITO'].astype(str).apply(unidecode.unidecode)

# Diccionario de reemplazo para letras acentuadas
acento_a_sin_acento = {
    'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
    'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'
}

def remover_tildes(texto):
    return re.sub(r'[áéíóúÁÉÍÓÚ]', lambda x: acento_a_sin_acento[x.group()], str(texto))

proveedor_data['DEPARTAMENTO'] = proveedor_data['DEPARTAMENTO'].apply(remover_tildes)
proveedor_data['PROVINCIA'] = proveedor_data['PROVINCIA'].apply(remover_tildes)
proveedor_data['DISTRITO'] = proveedor_data['DISTRITO'].apply(remover_tildes)

In [188]:
# ,'VALORTOTALACCIONES','FECHA_REGISTRO'
proveedor_clean = proveedor_data[['RUC_PROVEEDOR','ACT_PRIMARIA','ESTADO_PROVEEDOR',
                                  'CONDICION_PROVEEDOR','DEPARTAMENTO','PROVINCIA','DISTRITO']].drop_duplicates().reset_index(drop=True)

In [189]:
mapeo_valores = {
    'PROV. CONST. DEL CALLAO': 'CALLAO',
}
proveedor_clean['PROVINCIA'] = proveedor_clean['PROVINCIA'].replace(mapeo_valores)

In [191]:
proveedor_clean.shape

(12893439, 7)

In [192]:
len(proveedor_clean['RUC_PROVEEDOR'].unique())

12892820

In [193]:
proveedor_clean[proveedor_clean['RUC_PROVEEDOR'].duplicated(keep=False)].head()

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO
9493,10103774000,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,ACTIVO,HABIDO,JUNIN,HUANCAYO,EL TAMBO
29682,10103790170,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,ACTIVO,HABIDO,LIMA,LIMA,SAN JUAN DE LURIGANCHO
54119,10103773890,NaN,ACTIVO,HABIDO,CALLAO,CALLAO,CALLAO
55647,10103782910,OTRAS ACTIVIDADES DE SERVICIOS PERSONALES N.C.P.,ACTIVO,HABIDO,MOQUEGUA,MARISCAL NIETO,MOQUEGUA
65682,10103773870,TERMINACIÓN Y ACABADO DE EDIFICIOS,ACTIVO,HABIDO,SAN MARTIN,PICOTA,PICOTA


In [194]:
proveedor_clean[proveedor_clean["RUC_PROVEEDOR"]==10103774000]

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO
9493,10103774000,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,ACTIVO,HABIDO,JUNIN,HUANCAYO,EL TAMBO
4744768,10103774000,NaN,ACTIVO,HABIDO,PUNO,PUNO,ACORA


#### unir con ubigeo

In [195]:
df_ubigeo = pd.read_csv(folder_data+"ubigeo_distrito.csv")
df_ubigeo = df_ubigeo[['departamento','provincia','distrito','latitude','longitude']]
df_ubigeo.head()

,departamento,provincia,distrito,latitude,longitude
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,-6.229444,-77.872778
1,AMAZONAS,CHACHAPOYAS,ASUNCION,-6.032500,-77.710833
2,AMAZONAS,CHACHAPOYAS,BALSAS,-6.835833,-78.019722
3,AMAZONAS,CHACHAPOYAS,CHETO,-6.255556,-77.700833
4,AMAZONAS,CHACHAPOYAS,CHILIQUIN,-6.078333,-77.737500


In [196]:
# Asegúrate de que las columnas tienen el mismo nombre y formato en ambos DataFrames
# Cambiar los nombres de las columnas en 'proveedor_data' para que coincidan con 'ubigeo' si es necesario
df_ubigeo.rename(columns={'departamento': 'DEPARTAMENTO',
                               'provincia': 'PROVINCIA', 
                               'distrito': 'DISTRITO'}, inplace=True)

# Realizar el merge
master_proveedor = pd.merge(proveedor_clean, df_ubigeo, on=['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'], how='left')
master_proveedor.rename(columns={'latitude': 'LATITUDE', 'longitude': 'LONGITUDE'}, inplace=True)

In [197]:
master_proveedor.head()

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO,LATITUDE,LONGITUDE
0,10100275560,NaN,ACTIVO,NO HABIDO,UCAYALI,CORONEL PORTILLO,CALLERIA,-8.368056,-74.543333
1,10100793370,NaN,ACTIVO,HABIDO,TUMBES,TUMBES,TUMBES,-3.571111,-80.459167
2,10100970330,OTRAS ACTIVIDADES ESPECIALIZADAS DE LA CONSTRU...,ACTIVO,HABIDO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,-12.029722,-77.010000
3,10101299540,NaN,ACTIVO,HABIDO,TACNA,TACNA,TACNA,-18.001944,-70.251944
4,10101679950,NaN,ACTIVO,HABIDO,TACNA,TACNA,ALTO DE LA ALIANZA,-17.993056,-70.247778


In [198]:
master_proveedor[master_proveedor["LATITUDE"].isnull()].head()

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO,LATITUDE,LONGITUDE
711,20100608509,NaN,BAJA DE OFICIO,HABIDO,nan,nan,nan,NaN,NaN
1076,10160452124,NaN,ACTIVO,HABIDO,nan,nan,nan,NaN,NaN
1343,10208622167,OTRAS ACTIVIDADES DE SERVICIOS PERSONALES N.C.P.,ACTIVO,HABIDO,ICA,NASCA,MARCONA,NaN,NaN
1657,99103774060,NaN,NaN,NaN,nan,nan,nan,NaN,NaN
1658,99103774284,NaN,NaN,NaN,nan,nan,nan,NaN,NaN


In [199]:
master_proveedor[master_proveedor["LATITUDE"].isnull()].shape

(56388, 9)

#### unir con seace

In [200]:
%%time

seace_data = get_dataset('ds_seace',0)


Tamaños de archivos CSV en DS02_SEACE.zip:
['DS02_SEACE.csv']
DS02_SEACE.csv
CPU times: user 1.69 s, sys: 19.9 ms, total: 1.71 s
Wall time: 1.73 s


In [201]:
seace_data.head()

,N_ID_EXPEDE,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,TIPO_PROCESO,PROCESO_SELECCION,FECHA_PUBLICACION,FECHA_ADJUDICACION,TIPO_MONEDA,TIPO_CAMBIO_CONV,MONTO_ADJUDICADO,MONTO_ADJ_PROVEEDOR,NUMERO_ITEMS,RUC_PROVEEDOR,ES_CONSORCIO
0,680584,20555195444,SUPERINTENDENCIA NACIONAL DE FISCALIZACIÓN LAB...,GOBIERNO NACIONAL,TRABAJO Y PROMOCIÓN DEL EMPLEO,LIMA,LIMA,JESUS MARIA,Contratación Directa,DIRECTA-PROC-8-2020-SUNAFIL-1,15/12/2020,16/12/2020,S/,1.0,204000.00,204000.000,1,10101723050,N
1,653921,20602398103,REGION POLICIAL HUANUCO - SAN MARTIN - UCAYALI,GOBIERNO NACIONAL,INTERIOR,HUANUCO,HUANUCO,HUANUCO,Adjudicación Simplificada,AS-SM-6-2020-UEHSMU-4,29/09/2020,13/10/2020,S/,1.0,224913.00,224913.000,1,10102123960,N
2,590946,20207736261,PROYECTO ESPECIAL BINACIONAL DESARROLLO INTEGR...,GOBIERNO NACIONAL,AGRARIO Y DE RIEGO,LORETO,MAYNAS,IQUITOS,Adjudicación Simplificada,AS-SM-16-2019-PEBDICP/OEC-1,24/12/2019,30/01/2020,S/,1.0,72000.00,72000.000,1,10102395900,N
3,592653,20548776920,PROGRAMA NACIONAL DE SANEAMIENTO RURAL,GOBIERNO NACIONAL,VIVIENDA CONSTRUCCIÓN Y SANEAMIENTO,LIMA,LIMA,MIRAFLORES,Adjudicación Simplificada,AS-SM-71-2019-PNSR-1,30/12/2019,17/02/2020,S/,1.0,129756.44,77853.864,1,10102560950,S
4,633502,20148170933,MUNICIPALIDAD PROVINCIAL DE RIOJA,GOBIERNO LOCAL,GOBIERNO LOCAL,SAN MARTIN,RIOJA,RIOJA,Regímen Especial,RES-PROC-12-2020-MPR/CS-1,31/07/2020,14/08/2020,S/,1.0,65299.20,65299.200,1,10102560950,N


In [202]:
len(seace_data["RUC_PROVEEDOR"].unique())

81909

In [203]:
# normalizar la unidad monetaria (convertir todos los montos a soles)
seace_data["MONTO_ADJUDICADO_SOLES"] = seace_data["MONTO_ADJUDICADO"] * seace_data["TIPO_CAMBIO_CONV"]
seace_data["MONTO_ADJ_PROVEEDOR_SOLES"] = seace_data["MONTO_ADJ_PROVEEDOR"] * seace_data["TIPO_CAMBIO_CONV"]

In [204]:
proveedor_seace = seace_data.groupby(['RUC_PROVEEDOR']).agg( # 'NOMBRE_DEPARTAMENTO', 'NOMBRE_PROVINCIA', 'NOMBRE_DISTRITO'
    NRO_EXP=('N_ID_EXPEDE', 'nunique'),
    TOTAL_ADJ_PROVEEDOR=('MONTO_ADJ_PROVEEDOR_SOLES', 'sum')
).reset_index()

In [205]:
len(proveedor_seace['RUC_PROVEEDOR'].unique())

81909

In [206]:
proveedor_seace.head()

,RUC_PROVEEDOR,NRO_EXP,TOTAL_ADJ_PROVEEDOR
0,00103758451,1,2.663700e+06
1,05103759588,1,1.027540e+07
2,06103759579,1,7.175203e+05
3,06103759585,1,2.898375e+05
4,06103759589,1,1.772115e+04


In [207]:
proveedor_seace.rename(columns={'NOMBRE_DEPARTAMENTO': 'DEPARTAMENTO', 
                               'NOMBRE_PROVINCIA': 'PROVINCIA', 
                               'NOMBRE_DISTRITO': 'DISTRITO'}, inplace=True)

In [208]:
master_proveedor = master_proveedor.merge(proveedor_seace, on=['RUC_PROVEEDOR'], how='left')

In [209]:
master_proveedor.head()

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO,LATITUDE,LONGITUDE,NRO_EXP,TOTAL_ADJ_PROVEEDOR
0,10100275560,NaN,ACTIVO,NO HABIDO,UCAYALI,CORONEL PORTILLO,CALLERIA,-8.368056,-74.543333,NaN,NaN
1,10100793370,NaN,ACTIVO,HABIDO,TUMBES,TUMBES,TUMBES,-3.571111,-80.459167,NaN,NaN
2,10100970330,OTRAS ACTIVIDADES ESPECIALIZADAS DE LA CONSTRU...,ACTIVO,HABIDO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,-12.029722,-77.010000,NaN,NaN
3,10101299540,NaN,ACTIVO,HABIDO,TACNA,TACNA,TACNA,-18.001944,-70.251944,NaN,NaN
4,10101679950,NaN,ACTIVO,HABIDO,TACNA,TACNA,ALTO DE LA ALIANZA,-17.993056,-70.247778,NaN,NaN


In [210]:
master_proveedor.shape

(12893439, 11)

In [211]:
len(master_proveedor['RUC_PROVEEDOR'].unique())

12892820

In [212]:
master_proveedor.to_csv(folder_data+"master_proveedor.csv",index=False)

In [213]:
master_proveedor[master_proveedor['RUC_PROVEEDOR'].duplicated(keep=False)].shape

(1238, 11)

In [214]:
master_proveedor[master_proveedor['RUC_PROVEEDOR'].duplicated(keep=False)].head()

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO,LATITUDE,LONGITUDE,NRO_EXP,TOTAL_ADJ_PROVEEDOR
9493,10103774000,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,ACTIVO,HABIDO,JUNIN,HUANCAYO,EL TAMBO,-12.050278,-75.221389,NaN,NaN
29682,10103790170,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,ACTIVO,HABIDO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,-12.029722,-77.010000,NaN,NaN
54119,10103773890,NaN,ACTIVO,HABIDO,CALLAO,CALLAO,CALLAO,-12.063056,-77.146944,NaN,NaN
55647,10103782910,OTRAS ACTIVIDADES DE SERVICIOS PERSONALES N.C.P.,ACTIVO,HABIDO,MOQUEGUA,MARISCAL NIETO,MOQUEGUA,-17.194167,-70.933333,NaN,NaN
65682,10103773870,TERMINACIÓN Y ACABADO DE EDIFICIOS,ACTIVO,HABIDO,SAN MARTIN,PICOTA,PICOTA,-6.920000,-76.330278,NaN,NaN
